In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cambio = pd.read_excel('../datasets/tc_turistas.xlsx',
                       sheet_name=0)

turista = pd.read_excel('../datasets/tc_turistas.xlsx',
                        sheet_name=2)

# Limpieza de datos

## Cambio

In [3]:
cambio.rename(
    columns={
        'FECHA': 'fecha',
        'DOLAR Oficial': 'dolar_oficial',
        'DOLAR Blue': 'dolar_blue'
    },
    inplace=True
)

Para completar los múltiples datos faltantes de los precios decidimos usar el último valor observado.

Otra forma posible hubise sido usar alguna medida resumen para los valores faltantes, pero en este caso eso haría que fluctúe mucho el incremento del dólar, cuando en realidad tiene una tendencia creciente.

In [4]:
cambio.dolar_oficial.fillna(method='ffill', inplace=True)
cambio.dolar_blue.fillna(method='ffill', inplace=True)

Agrupamos por mes y calculamos el promedio de los valores.

In [5]:
cambio_mensual = cambio.set_index('fecha').resample('M').mean()

In [6]:
cambio_mensual.index = cambio_mensual.index.strftime('%Y-%m')

In [7]:
cambio_mensual

,dolar_oficial,dolar_blue
fecha,,
2017-06,16.293333,16.498000
2017-07,17.375484,17.169355
2017-08,17.626290,18.218065
2017-09,17.428333,17.948333
2017-10,17.661290,17.963871
2017-11,17.693333,18.047333
2017-12,17.950000,18.305484
2018-01,19.267742,19.652258
2018-02,20.085714,20.157143


## Turista

In [8]:
turista.rename(
    columns={col: col.lower() for col in turista.columns},
    inplace=True
)

Casteamos las columnas de los turistas llegados de cada país a `int`.

In [9]:
turista[turista.columns[3:]] =\
    turista[turista.columns[3:]].astype('int64')

Viendo los máximos de cada campo vemos que hay un valor atípico en uno de ellos.

In [10]:
turista.max()

anio                   2021
mes                     sep
date                  44256
bolivia              187955
brasil               242582
chile       571838802363218
paraguay             138972
uruguay               86374
dtype: object

El valor atípico lo tiene la entrada de año 2018 y mes abril.

In [11]:
turista[turista.chile == 571838802363218]

,anio,mes,date,bolivia,brasil,chile,paraguay,uruguay
63,2018,abr,43191,87508,77916,571838802363218,57205,34821


Decidimos tomar el promedio del año previo y sucesor de 2018 para reemplazar el valor atípico.

In [12]:
turista.loc[turista.chile == 571838802363218, 'chile'] = \
    turista[(turista.anio.isin([2017, 2019])) & (turista.mes == 'abr')]\
        .chile.mean()\
        .astype('int64')

Mapeamos cada uno de los meses a su correspondiente número.

In [13]:
map_meses = { 
    'ene': '01',
    'feb': '02',
    'mar': '03',
    'abr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'ago': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dic': '12'
}

In [14]:
turista.mes = turista.mes.apply(lambda x: map_meses[x])

In [19]:
turista['fecha'] = \
    turista.anio.astype('str') + '-' + turista.mes

In [29]:
pd.merge(cambio_mensual, turista, left_index=True, right_on='fecha', how='left')

,dolar_oficial,dolar_blue,anio,mes,date,bolivia,brasil,chile,paraguay,uruguay,fecha
53,16.293333,16.498000,2017,06,42887,91354,44950,51901,44105,22149,2017-06
54,17.375484,17.169355,2017,07,42917,139655,67292,90150,74440,25872,2017-07
55,17.626290,18.218065,2017,08,42948,101980,53708,67773,60127,26348,2017-08
56,17.428333,17.948333,2017,09,42979,113370,79188,66416,70244,20974,2017-09
57,17.661290,17.963871,2017,10,43009,108988,86278,74250,65150,27705,2017-10
58,17.693333,18.047333,2017,11,43040,100106,73925,67212,58810,41117,2017-11
59,17.950000,18.305484,2017,12,43070,113630,106258,162180,54931,53165,2017-12
60,19.267742,19.652258,2018,01,43101,156813,124143,98232,47638,72066,2018-01
61,20.085714,20.157143,2018,02,43132,96556,170947,80980,49857,56729,2018-02
62,20.480645,20.743226,2018,03,43160,77099,87156,83496,66549,54561,2018-03


### *Preguntar que es la columna date de turista, que hacer con el valor atípico de 2018-abr-chile y por que en el intervalo de cuarenta date no es 0*

# Exploración de datos